# Coach Pat Cavanaugh: Leading and Teaming

## Coach Pat spent 12 years as an elementary and middle school principal in Metro Detroit. His expertise in school climate and culture, staff empowerment and community involvement led to high-performing student achievement and increased enrollment. Pat also spent time as a classroom teacher and college football coach. Throughout his career, Pat has been a frequent guest speaker for various businesses, schools, athletic teams, and higher education institutions.

## 🔥 Let's get the required packages

In [ ]:
#r "nuget: Azure.AI.OpenAI, 2.0.0-beta.4"
#r "nuget: Azure.Core, 1.42.0"
#r "nuget: Azure.Identity, 1.13.0-beta.1"
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.18.0-rc"
#r "nuget: Microsoft.SemanticKernel.Abstractions, 1.18.1-rc"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.18.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.OpenAI, 1.18.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.18.1-rc"
#r "nuget: Microsoft.SemanticKernel.Core, 1.18.1-rc"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.18.1-alpha"
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"

## 🔑 And our keys ...

My config.json looks like this:
```
{
  "type": "azure",
  "model": "gpt-4o",
  "endpoint": "https://ai-johnmaeda.../",
  "apikey": "...",
  "org": "",
  "asst1": "asst_...",
  "asst2": "asst_...",
}
```

In [ ]:
#!import ../config/MaedaSettings.cs
#!import ../config/Utils.cs

using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Plugins.Core;
using System;
using System.Text.Json;
using System.Collections.Generic;
using System.Threading.Tasks;
using System.ComponentModel;
using System.Net.Http;
using System.Net.Http.Headers;
using Microsoft.Extensions.DependencyInjection;
using System.Threading;
using OpenAI.Files;
using OpenAI.VectorStores;

var settingsPath = Path.GetFullPath("../config/settings.json");
Console.WriteLine($"Settings Path: {settingsPath}");

var maedaSettings = new MaedaSettings(settingsPath, true);

// Load all settings into a tuple
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = maedaSettings.LoadSettings();

# 🥸 Make an Azure AI Agent wrapper

In [13]:
#!import ../config/Utils.cs

#pragma warning disable SKEXP0110, SKEXP0001, SKEXP0050, CS8600, CS8604, OPENAI001

public class MyAzureAIAgent
{
    private readonly string _assistantId;
    private readonly string _deploymentName;
    private readonly string _endpoint;
    private readonly string _apiKey;
    private string? _threadId;
    private OpenAIAssistantAgent? _assistantAgent;
    public string namae;

    public MyAzureAIAgent(string deploymentName, string endpoint, string apiKey, string assistantId, List<object>? plugins = null)
    {
        _assistantId = assistantId;
        _deploymentName = deploymentName;
        _endpoint = endpoint;
        _apiKey = apiKey;

        // Initialize the agent with plugins when constructing
        Task.Run(async () => await InitializeAgentWithPluginsAndThread(plugins)).GetAwaiter().GetResult();

    }

    public OpenAIAssistantAgent GetAgent()
    {
        return _assistantAgent;
    }
    
    // add an optional list of plugin objects as type new List<object>
    public async Task InitializeAgentWithPluginsAndThread(List<object>? plugins = null)
    {
        IKernelBuilder builder = Microsoft.SemanticKernel.Kernel.CreateBuilder();
        builder.AddAzureOpenAIChatCompletion(
                        deploymentName: _deploymentName,
                        endpoint: _endpoint,
                        apiKey: _apiKey);
        if (plugins != null)
        {
            foreach (var plugin in plugins)
            {
                // say which pluign you're adding by printing the type
                Console.WriteLine($"Adding plugin: {plugin.GetType()}");
                builder.Plugins.AddFromObject(plugin);
            }
        }
        Microsoft.SemanticKernel.Kernel kernel = builder.Build();

        OpenAIClientProvider provider = OpenAIClientProvider.ForAzureOpenAI(_apiKey, new Uri(_endpoint));

        _assistantAgent = await OpenAIAssistantAgent.RetrieveAsync(kernel, provider, _assistantId);
        
        namae = _assistantAgent.Name;

        _threadId = await _assistantAgent.CreateThreadAsync();
        Console.WriteLine($"Agent {namae} with ID {Utils.ObfuscateString(_assistantId)} initialized.");
    }

    public async Task<string> AskAgent(string question)
    {
        if (_assistantAgent == null || _threadId == null)
        {
            throw new InvalidOperationException("Agent not initialized. Call InitializeAgent first.");
        }

        // Add the user's question to the chat.
        await _assistantAgent.AddChatMessageAsync(_threadId, new ChatMessageContent(AuthorRole.User, question));

        // Retrieve the response.
        string responseText = "";
        await foreach (ChatMessageContent content in _assistantAgent.InvokeAsync(_threadId))
        {
            if (content.Role != AuthorRole.Tool)
            {
                responseText += content.Content;
            }
        }

        return responseText; // Return the response as a string
    }

    public async Task EndSession()
    {
        if (_assistantAgent != null && _threadId != null)
        {
            bool result = await _assistantAgent.DeleteThreadAsync(_threadId);
            if (result)
            {
                _threadId = null;
            }
            else
            {
                Console.WriteLine("Failed to delete the thread.");
            }
        }
    }
}


## ☎️ And a way to "talk" to @agents

In [14]:
#!import ../config/Utils.cs 

#pragma warning disable SKEXP0110, SKEXP0001, SKEXP0050, CS8600, CS8604, OPENAI001

public string CreateAtName(string name)
{
    return name.Replace(" ", "").ToLower();
}

public async Task RunIndependentAgentsSession(List<MyAzureAIAgent> myagents)
{
    // Dictionary to hold initialized agents (mapped by processed agent names)
    var agents = new Dictionary<string, MyAzureAIAgent>(StringComparer.OrdinalIgnoreCase);
    string? defaultAgentName = null;

    try
    {
        Console.WriteLine("Initializing agents...");
        // Instantiate and initialize each agent from the list of assistant IDs
        foreach (var agent in myagents)
        {
            Console.WriteLine($"Initializing agent {agent.namae}...");
            // Process the namae to remove spaces and lowercase it
            string processedNamae = CreateAtName(agent.namae);

            // Add agent to dictionary with the processed name as the key
            agents[processedNamae] = agent;

            // Set the first agent as the default if it's not already set
            if (defaultAgentName == null)
            {
                defaultAgentName = processedNamae;
            }
        }

        // Ensure we have a default agent
        if (defaultAgentName == null)
        {
            throw new InvalidOperationException("No default agent found. Please check your assistant IDs.");
        }

        string input;
        do
        {
            // Print available agent names with an at sign in front of each of them and all on a single line
            // accrue them and include the at sign in fron tof them
            string availableAgents = "";
            foreach (var agentName in agents.Keys)
            {
                availableAgents += $"@{agentName} ";
            }

            // Get user input
            input = await InteractiveKernel.GetInputAsync($"Ask {availableAgents} (type '/bye' to exit):");

            if (input.ToLower() != "/bye")
            {
                // By default, use the default agent
                string selectedAgentName = defaultAgentName;

                // Check if there's an @mention in the input
                foreach (var agentName in agents.Keys)
                {
                    if (input.ToLower().Contains($"@{agentName}"))
                    {
                        selectedAgentName = agentName;

                        // Remove the @mention from the input
                        input = input.Replace($"@{agentName}", "").Trim();
                        break;
                    }
                }

                // Get the selected agent
                var selectedAgent = agents[selectedAgentName];
                string response = await selectedAgent.AskAgent(input);
                Console.WriteLine(Utils.WordWrap($"@{selectedAgentName}: {response}", 80));
            }

        } while (input.ToLower() != "/bye");
    }
    finally
    {
        // Clean up and end each agent's session
        foreach (var agent in agents.Values)
        {
            await agent.EndSession();
        }
    }
}


## 🔥 Make a kernel

In [15]:
#pragma warning disable SKEXP0110, SKEXP0001, SKEXP0050, CS8600, CS8604, OPENAI001

IKernelBuilder builder = Microsoft.SemanticKernel.Kernel.CreateBuilder();
builder.AddAzureOpenAIChatCompletion(
                deploymentName: model,
                endpoint: azureEndpoint,
                apiKey: apiKey);

Microsoft.SemanticKernel.Kernel kernel = builder.Build();

OpenAIClientProvider provider = OpenAIClientProvider.ForAzureOpenAI(apiKey, new Uri(azureEndpoint));

## 🌱 Let's sprout an @agent in code

In [ ]:
#!import ../config/Utils.cs
#!import ../config/MaedaAgentManager.cs

MaedaAgentManager agentManager = new MaedaAgentManager(kernel, provider);

string agentName = "coachpat";
string description = """
  Coach Pat spent 12 years as an elementary and middle school principal in Metro 
  Detroit. His expertise in school climate and culture, staff empowerment and 
  community involvement led to high-performing student achievement and increased 
  enrollment. Pat also spent time as a classroom teacher and college football coach. 
  Use Green Bay Packers stories when appropriate.
  """;
string instructions = """
  Having spent 12 years as an elementary and middle school principal in Metro 
  Detroit, use your expertise in school climate and culture, staff empowerment and 
  community involvement led to high-performing student achievement and increased 
  enrollment. You also spent time as a classroom teacher and college football coach. 
  Throughout your career, you have been a frequent guest speaker for various businesses, 
  schools, athletic teams, and higher education institutions.
  Assist your mentee by unlocking their leadership potential by nurtures leadership 
  skills, cultivates a positive culture, and empowers you to achieve your goals 
  while positively influencing those around you.
  Be concise and don't say more than 3 sentences at a time. 
  Tell practical short stories to help your mentee find the leader in themselves.
  Always answer in a conversational manner. Encourage your mentee to speak up.
  Come at coaching from how you can help people grow and become better leaders.  
  This translates to your work as a teacher, principal, and football coach.
""";
List<string> filePaths = new List<string> { "coachnotes.txt" };

string coachId = await agentManager.CreateAgentWithFilesAsync(agentName, model, description, instructions, filePaths);
Console.WriteLine($"The agent {Utils.ObfuscateString(coachId)} has been minted. Refer to @{CreateAtName(agentName)} in chat.");
Console.WriteLine($"Warning: The agent {coachId} has files and a vector store attached to it that may incur a running cost if not deleted.");


## 📞 Let's give Coach a call ...

In [ ]:
#pragma warning disable SKEXP0110, SKEXP0001, SKEXP0050, CS8600, CS8604, OPENAI001

// make MyAzureAIAgent
var agent1 = new MyAzureAIAgent(model, azureEndpoint, apiKey, coachId);

// Call the session runner and pass the list of assistant IDs inline
await RunIndependentAgentsSession(
    new List<MyAzureAIAgent>
    {
        agent1
    }
);


## 🧽 Be sure to clean up 

In [ ]:
await agentManager.DeleteAssociatedAgentFilesAndVectorStoreAsync(coachId);

In [ ]:
await agentManager.DeleteAllAgentsAsync();